In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_wmi_dcerpc_wmi_IWbemServices_ExecMethod.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','SubjectUserName','TargetUserName','TargetLogonId','NewProcessName','CommandLine']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4688)
    & (df['ParentProcessName'].str.lower().str.endswith('wmiprvse.exe', na=False))
    & (df['TargetLogonId'] != '0x3e7')
]
.head()
)

,@timestamp,Hostname,SubjectUserName,TargetUserName,TargetLogonId,NewProcessName,CommandLine
551,2020-09-21T04:14:55.136Z,WORKSTATION6.theshire.local,WORKSTATION6$,pgustavo,0x32c5cb5,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,C:\Windows\System32\WindowsPowershell\v1.0\pow...


In [4]:
(
df[['@timestamp','Hostname','User','Image','CommandLine']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 1)
    & (df['ParentImage'].str.lower().str.endswith('wmiprvse.exe', na=False))
    & (df['LogonId'] != '0x3e7')
]
.head()
)

,@timestamp,Hostname,User,Image,CommandLine
368,2020-09-21T04:14:55.050Z,WORKSTATION6.theshire.local,THESHIRE\pgustavo,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,C:\Windows\System32\WindowsPowershell\v1.0\pow...


In [9]:
processCreateDf = (
# (
df[['@timestamp','Hostname','SubjectUserName','TargetUserName','TargetLogonId','NewProcessName','CommandLine','IpAddress']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4688)
    & (df['ParentProcessName'].str.lower().str.endswith('wmiprvse.exe', na=False))
    & (df['TargetLogonId'] != '0x3e7')
]
.head()
)

networkLogonDf = (
# (
df[['@timestamp', 'Hostname', 'TargetUserName', 'TargetLogonId', 'IpAddress']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4624)
    & (df['LogonType'] == 3)
    & (~df['SubjectUserName'].str.endswith('.*$', na=False))
]
)

(
pd.merge(processCreateDf, networkLogonDf,
    on = 'TargetLogonId', how = 'inner')
)


,@timestamp_x,Hostname_x,SubjectUserName,TargetUserName_x,TargetLogonId,NewProcessName,CommandLine,IpAddress_x,@timestamp_y,Hostname_y,TargetUserName_y,IpAddress_y
0,2020-09-21T04:14:55.136Z,WORKSTATION6.theshire.local,WORKSTATION6$,pgustavo,0x32c5cb5,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,C:\Windows\System32\WindowsPowershell\v1.0\pow...,NaN,2020-09-21T04:14:55.119Z,WORKSTATION6.theshire.local,pgustavo,172.18.39.5
